## Importing Modules

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3, vgg16, vgg19 # For Transfer Learing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Data Augmentation 

In [ ]:
# Checking the status of GPY
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2183173151546414160
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14465892352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11720209517877299104
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
# # open -> 40,000 images 
# # close -> 40,000 images  
# test -> 2002 images i.e. 5% of data
# Train -> 80 % data for training and 20 % data for validation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
batch_size = 8

In [ ]:
# Data Augumentation for Train images means for one image in train set it will generate more 5 images of it.

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range = 0.2, 
                                   shear_range=0.2,
                                   zoom_range = 0.2, 
                                   width_shift_range = 0.2, 
                                   height_shift_range = 0.2,
                                   validation_split = 0.2)

train_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/dataFinal/train',
                                              target_size=(80,80),
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              subset = 'training')

validation_data = train_datagen.flow_from_directory(r'/content/drive/MyDrive/dataFinal/train',
                                              target_size=(80,80),
                                              batch_size = batch_size,
                                              class_mode = 'categorical',
                                              subset = 'validation')

# target size means images 

Found 1440 images belonging to 2 classes.
Found 360 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'/content/drive/MyDrive/dataFinal/train',
                                            target_size = (80,80),
                                            batch_size = batch_size, 
                                            class_mode = 'categorical')

Found 1800 images belonging to 2 classes.


In [ ]:
# bmodel i.e base model Inception V3
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output

hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [16]:
# checkpoint of folder , verbose progress bar
checkpoint = ModelCheckpoint(r'/content/drive/MyDrive/dataFinal/models/model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(train_data,
          steps_per_epoch=train_data.samples//batch_size,
          validation_data=validation_data,
          validation_steps=validation_data.samples//batch_size,
          callbacks=callbacks,
          epochs=5)

Epoch 1/5
180/180 [==============================] - ETA: 0s - loss: 0.0527 - accuracy: 0.9826
Epoch 1: val_loss improved from inf to 0.03921, saving model to /content/drive/MyDrive/dataFinal/models/model.h5
180/180 [==============================] - 15s 58ms/step - loss: 0.0527 - accuracy: 0.9826 - val_loss: 0.0392 - val_accuracy: 0.9889 - lr: 0.0010
Epoch 2/5
180/180 [==============================] - ETA: 0s - loss: 0.0671 - accuracy: 0.9771
Epoch 2: val_loss improved from 0.03921 to 0.02581, saving model to /content/drive/MyDrive/dataFinal/models/model.h5
180/180 [==============================] - 9s 53ms/step - loss: 0.0671 - accuracy: 0.9771 - val_loss: 0.0258 - val_accuracy: 0.9944 - lr: 0.0010
Epoch 3/5
180/180 [==============================] - ETA: 0s - loss: 0.0493 - accuracy: 0.9903
Epoch 3: val_loss improved from 0.02581 to 0.00136, saving model to /content/drive/MyDrive/dataFinal/models/model.h5
180/180 [==============================] - 9s 51ms/step - loss: 0.0493 - accu

# Model Evalution

In [20]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

180/180 [==============================] - 7s 37ms/step - loss: 0.0254 - accuracy: 0.9917
0.025421960279345512
0.9916666746139526


In [24]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)

225/225 [==============================] - 5s 24ms/step - loss: 0.0088 - accuracy: 0.9978
0.025421960279345512
0.9916666746139526
